In [4]:
import warnings
import pandas as pd
import os
import re
from datetime import datetime
import numpy as np
warnings.filterwarnings('ignore')
%matplotlib inline
#https://discord.com/channels/1134059900666916935/1283610000484208670
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.manifold import TSNE

In [5]:
# Define input and output folders
input_folder = "data/txt"
output_folder = "data/output"
file_paths = [os.path.join(input_folder, file) for file in os.listdir(input_folder) if file.endswith(".txt")]

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Job dictionary and spec ranges
job_dict = {
    "허": "허밋", "시프": "시프", "썬": "썬콜", "불독": "불독",
    "프": "프리스트", "레": "레인저", "저": "저격수",
    "용": "용기사", "크": "크루세이더", "나": "나이트"
}
job_spec_ranges = {
    "용": (3000, 9000), "크": (2000, 9000), "나": (2000, 9000),
    "허": (1500, 4000), "시프": (1500, 5000), "썬": (500, 1200),
    "불독": (500, 1200), "프": (500, 1200), "레": (2000, 9000),
    "저": (2000, 9000)
}
level_min, level_max = 80, 190

# Regex patterns
time_pattern = r"(오전|오후) \d{1,2}:\d{2}"
job_pattern = r"(\d{2,3})\s?(" + "|".join(job_dict.keys()) + r")"
spec_pattern = r"(\d{3,4})"
map_pattern = r"(망용둥|위둥|남둥|큰둥|와협|블와둥|협동|레와둥|붉켄|검켄|푸켄|불어전|물어전|오징어|깊바협|망둥쩔|듀파|듀미굴|갈림길|산양|하둥)"

In [6]:
def extract_level_and_spec(text, job_start, job_end, level_range, spec_range):
    left_text = text[job_start-5 : job_start+5]
    right_text = text[job_end+1 : job_end + 10]
    
    level, spec = None, None

    # Search for level in the left_text
    for match in re.finditer(r"\d{2,3}", left_text[::-1]):  # Reverse for easier parsing
        num = int(match.group()[::-1])  # Reverse back the number
        if level_range[0] <= num <= level_range[1]:
            level = num
            break

    # Search for spec in the right_text
    for match in re.finditer(r"\d{3,4}", right_text):
        num = int(match.group())
        if spec_range[0] <= num <= spec_range[1]:
            spec = num
            break

    return level, spec

In [7]:
def collect_initial_data(file_path):
    columns = ["party_id", "time", "level", "job", "spec", "map", "date", "valid", "valid_spec"]
    df = pd.DataFrame(columns=columns)

    # Extract date from filename
    month = file_path[-8:-6]
    day = file_path[-6:-4]
    date_from_file = f"2024-{month}-{day}"
    
    party_id = 1
    
    with open(file_path, "r", encoding="utf-8") as file:
        data = file.read()

    blocks = data.split("오늘")
    for block in blocks[1:]:
        time_match = re.search(time_pattern, block)
        time = time_match.group(0) if time_match else None
        map_match = re.search(map_pattern, block)
        map_name = map_match.group(0) if map_match else None

        job_matches = list(re.finditer(job_pattern, block))
        for job_match in job_matches:
            job_short = job_match.group(2)
            job_full = job_dict.get(job_short, "Unknown")

            level, spec = extract_level_and_spec(
                block, job_match.start(), job_match.end(),
                (level_min, level_max), job_spec_ranges.get(job_short, (None, None))
            )
            
            valid = map_name is not None and job_full != "Unknown"

            # print(level, job_full, spec)
            
            # Append new row to DataFrame
            df = pd.concat([
                df,
                pd.DataFrame([{
                    "party_id": party_id,
                    "time": time,
                    "level": level,
                    "job": job_full,
                    "spec": spec,
                    "map": map_name,
                    "date": date_from_file,
                    "valid": valid,
                    "valid_spec": None
                }])
            ], ignore_index=True)

        party_id += 1

    # Remove if level is NaN
    df = df.dropna(subset=['level'])
    # Remove duplicates
    df = df.drop_duplicates(subset=["map", "level", "job", "spec"], keep="first")

    #스펙 결측치 처리
    # Step 1: Calculate spec_by_level, handling NaN values for spec and level
    df["spec_by_level"] = df.apply(
        lambda row: row["spec"] / row["level"] if pd.notna(row["spec"]) and pd.notna(row["level"]) else None,
        axis=1
    )
    
    # Step 2: Calculate average spec_by_level by job
    avg_spec_by_level = df.groupby("job")["spec_by_level"].mean().rename("avg_spec_by_level")
    
    # Step 3: Merge the avg_spec_by_level back into the dataframe
    df = df.merge(avg_spec_by_level, on="job", how="left")
    
    # Step 4: Fill NaN in spec_by_level with the average spec_by_level for the job
    df["spec_by_level"].fillna(df["avg_spec_by_level"], inplace=True)
    
    # Step 5: Determine spec_valid based on 20% margin from avg_spec_by_level
    def is_spec_valid(row):
        if pd.notna(row["spec"]):  # Only proceed if spec is not NaN
            if pd.notna(row["spec_by_level"]) and pd.notna(row["avg_spec_by_level"]):
                lower_bound = 0.8 * row["avg_spec_by_level"]
                upper_bound = 1.2 * row["avg_spec_by_level"]
                return lower_bound <= row["spec_by_level"] <= upper_bound
        return False  # Return False if spec is NaN or not within the range
    
    # Step 6: Apply the is_spec_valid function to the dataframe
    df["valid_spec"] = df.apply(is_spec_valid, axis=1)
    
    # Step 7: Create spec_filled: if spec is NaN, fill with level * avg_spec_by_level, else keep spec as is
    df["spec_filled"] = df.apply(
        lambda row: row["spec"] if pd.notna(row["spec"]) else (
            row["level"] * row["avg_spec_by_level"] if pd.notna(row["level"]) and pd.notna(row["avg_spec_by_level"]) else None
        ),
        axis=1
    )
    
    # Step 8: Calculate the average distance between spec and avg_spec_by_level for each job
    df["spec_distance"] = df.apply(
        lambda row: abs(row["spec"] - row["avg_spec_by_level"]) if pd.notna(row["spec"]) and pd.notna(row["avg_spec_by_level"]) else None,
        axis=1
    )
    
    # Step 9: Calculate the average distance for each job
    job_avg_distances = df.groupby("job")["spec_distance"].mean().rename("avg_distance_per_job")
    
    # Step 10: Merge the average distance per job back into the dataframe
    df = df.merge(job_avg_distances, on="job", how="left")

    JITTER_PERCENTAGE = 0.03
    # Step 11: Apply jitter based on the average distance per job
    # Apply jitter ONLY if spec_filled is created (i.e., spec is NaN)
    df["spec_filled"] = df.apply(
    lambda row: row["spec_filled"] + np.random.uniform(-row["spec_filled"] * JITTER_PERCENTAGE, row["spec_filled"] * JITTER_PERCENTAGE)
    if pd.isna(row["spec"]) and pd.notna(row["spec_filled"]) else row["spec_filled"],  # Apply jitter only when spec is missing
    axis=1
    )

    df.drop(columns=['spec_distance', 'avg_distance_per_job'], inplace=True)
    
    output_file = os.path.join(output_folder, os.path.basename(file_path).replace('.txt', '.csv'))
    df.to_csv(output_file, index=False)
    print(f"Data saved as {output_file}")
    return df

In [8]:
def process_data(df, file_path):
    """
    Filters and processes data to save only rows where valid and valid_spec are True.
    """
    # Filter rows where both 'valid' and 'valid_spec' are True
    processed_df = df[(df["valid"]) & (df["valid_spec"])]

    # Save the processed DataFrame to a CSV file
    processed_output_file = os.path.join(output_folder, os.path.basename(file_path).replace('.txt', '_processed.csv'))
    processed_df.to_csv(processed_output_file, index=False)

    print(f"Processed valid data saved as {processed_output_file}")
    return processed_df

In [9]:
def drop_level_outlier(df, level_percentile=99.99):
    # Filter out rows where 'level' is NaN
    df_level_not_na = df[df['level'].notna()]

    # Calculate the upper and lower level thresholds
    upper_level_threshold = np.percentile(df_level_not_na['level'], level_percentile)  # Top x% threshold
    lower_level_threshold = np.percentile(df_level_not_na['level'], 100 - level_percentile)  # Bottom x% threshold

    # Identify the outliers: those that are above the upper threshold or below the lower threshold
    outliers = df_level_not_na[(df_level_not_na['level'] >= upper_level_threshold) | (df_level_not_na['level'] <= lower_level_threshold)]

    # Print the thresholds and outliers for verification
    print(f"Upper level threshold (top {level_percentile}%): {upper_level_threshold}")
    print(f"Lower level threshold (bottom {100 - level_percentile}%): {lower_level_threshold}")
    print("Level Outliers:")
    print(outliers)

    # Drop the outliers from the original dataframe
    df = df.drop(outliers.index)

    return df

In [10]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd

# Step 1: Preprocess the data for t-SNE
def preprocess_data_for_tsne(df, numerical_features=["level", "spec_filled"], categorical_features=["job", "map"]):
    """
    Preprocesses the data for t-SNE:
    - Encodes categorical features (e.g., job, map).
    - Imputes missing values for numerical features (not filling with 0).
    - Normalizes numerical features.
    """
    # One-Hot Encode categorical features (e.g., job, map)
    encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    encoded_cats = encoder.fit_transform(df[categorical_features])
    encoded_df = pd.DataFrame(encoded_cats, columns=encoder.get_feature_names_out(categorical_features))

    # Impute missing values for numerical columns with the mean (instead of filling with 0)
    imputer = SimpleImputer(strategy='mean')
    df[numerical_features] = imputer.fit_transform(df[numerical_features])  # Impute missing numerical values

    # Normalize numerical features (including spec_filled)
    scaler = StandardScaler()
    normalized_nums = scaler.fit_transform(df[numerical_features])
    normalized_df = pd.DataFrame(normalized_nums, columns=numerical_features)

    # Combine numerical and categorical features
    combined_df = pd.concat([normalized_df, encoded_df], axis=1)
    return combined_df

# Step 2: Apply t-SNE to the data
def apply_tsne(df, perplexity=30, learning_rate=200, n_iter=1000):
    """
    Apply t-SNE to the dataset and return the 2D results.
    """
    # Preprocess the data
    processed_data = preprocess_data_for_tsne(df)

    # Apply t-SNE
    tsne = TSNE(n_components=2, perplexity=perplexity, learning_rate=learning_rate, n_iter=n_iter, random_state=42)
    tsne_results = tsne.fit_transform(processed_data)

    # Add t-SNE results to the DataFrame
    df["tsne_dim1"] = tsne_results[:, 0]
    df["tsne_dim2"] = tsne_results[:, 1]

    return df

# Step 3: Identify outliers based on t-SNE results and filter the top 0.1%
def filter_outliers_by_tsne(df, top_percentile=99.9):
    """
    Filter out the top n% (e.g., 0.1%) outliers based on t-SNE distance.
    """
    # Using NearestNeighbors to compute distances between points
    neighbors = NearestNeighbors(n_neighbors=2)  # Find the closest point (itself + 1 neighbor)
    neighbors.fit(df[["tsne_dim1", "tsne_dim2"]])
    distances, _ = neighbors.kneighbors(df[["tsne_dim1", "tsne_dim2"]])

    # Calculate the distance from the nearest neighbor
    df["distance_to_nearest"] = distances[:, 1]  # distance to the second nearest (ignoring self)

    # Calculate the threshold for the top n% outliers
    distance_threshold = np.percentile(df["distance_to_nearest"], top_percentile)
    print(f"Filtering out the top {top_percentile}% outliers with distance greater than {distance_threshold}")

    # Identify the outliers (rows that are filtered out)
    outliers = df[df["distance_to_nearest"] > distance_threshold]
    
    # Filter out the outliers from the original df
    df_filtered = df[df["distance_to_nearest"] <= distance_threshold]

    return df_filtered, outliers

# Example usage:
def apply_tsne_oulier_drop(df, top_percentile=99.9):
    df_with_tsne = apply_tsne(df)  # Apply t-SNE
    df_filtered, outliers = filter_outliers_by_tsne(df_with_tsne, top_percentile=99.9)  # Filter out top 0.1% outliers
    df = df.drop(outliers.index)
    print("tsne outliers:", outliers)
    return df

In [11]:
def process_files(file_paths, output_folder):
    all_raw_data = []
    all_processed_data = []

    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    for file_path in file_paths:
        # Collect initial raw data
        raw_df = collect_initial_data(file_path)

        # Process and filter valid data
        processed_df = process_data(raw_df, file_path)

        all_raw_data.append(raw_df)
        all_processed_data.append(processed_df)

    # Combine all raw DataFrames
    combined_raw_df = pd.concat(all_raw_data, ignore_index=True)
    combined_processed_df = pd.concat(all_processed_data, ignore_index=True)

    # level outlier drop
    # combined_raw_df = drop_level_outlier(combined_raw_df, level_percentile=99.99)
    # combined_processed_df = drop_level_outlier(combined_processed_df, level_percentile=99.999)
    
    # tsne outlier drop
    # combined_raw_df = apply_tsne_oulier_drop(combined_raw_df, top_percentile=99.99)
    # combined_processed_df = apply_tsne_oulier_drop(combined_processed_df, top_percentile=99.999)
    
    # Save the combined DataFrames
    combined_raw_output = os.path.join(output_folder, "df.csv")
    combined_processed_output = os.path.join(output_folder, "processed_df.csv")

    combined_raw_df.to_csv(combined_raw_output, index=False)
    combined_processed_df.to_csv(combined_processed_output, index=False)

    print(f"Combined raw data saved as '{combined_raw_output}'")
    print(f"Combined processed data saved as '{combined_processed_output}'")

    return combined_raw_df, combined_processed_df

In [12]:
# Process files and save results
df, processed_df = process_files(file_paths, output_folder)

Data saved as data/output/1219.csv
Processed valid data saved as data/output/1219_processed.csv
Data saved as data/output/1218.csv
Processed valid data saved as data/output/1218_processed.csv
Data saved as data/output/1220.csv
Processed valid data saved as data/output/1220_processed.csv
Data saved as data/output/1221.csv
Processed valid data saved as data/output/1221_processed.csv
Data saved as data/output/1222.csv
Processed valid data saved as data/output/1222_processed.csv
Data saved as data/output/1213.csv
Processed valid data saved as data/output/1213_processed.csv
Data saved as data/output/1212.csv
Processed valid data saved as data/output/1212_processed.csv
Data saved as data/output/1211.csv
Processed valid data saved as data/output/1211_processed.csv
Data saved as data/output/1215.csv
Processed valid data saved as data/output/1215_processed.csv
Data saved as data/output/1214.csv
Processed valid data saved as data/output/1214_processed.csv
Data saved as data/output/1216.csv
Proce

In [13]:
df

,party_id,time,level,job,spec,map,date,valid,valid_spec,spec_by_level,avg_spec_by_level,spec_filled
0,1,오전 12:33,88,허밋,None,붉켄,2024-12-19,True,False,18.446821,18.446821,1610.267049
1,1,오전 12:33,101,레인저,None,붉켄,2024-12-19,True,False,22.312474,22.312474,2212.530949
2,2,오전 12:33,90,허밋,None,붉켄,2024-12-19,True,False,18.446821,18.446821,1626.593477
3,2,오전 12:33,99,허밋,None,붉켄,2024-12-19,True,False,18.446821,18.446821,1773.072308
4,2,오전 12:33,105,썬콜,650,붉켄,2024-12-19,True,True,6.190476,6.212019,650.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
3930,292,오후 1:59,143,용기사,5000,블와둥,2024-12-17,True,True,34.965035,35.282984,5000.000000
3931,297,오후 1:59,119,프리스트,None,검켄,2024-12-17,True,False,6.320755,6.320755,732.841151
3932,297,오후 1:59,134,프리스트,None,검켄,2024-12-17,True,False,6.320755,6.320755,841.085808
3933,300,오후 1:59,123,프리스트,None,None,2024-12-17,False,False,6.320755,6.320755,784.703748


In [14]:
processed_df

,party_id,time,level,job,spec,map,date,valid,valid_spec,spec_by_level,avg_spec_by_level,spec_filled
0,2,오전 12:33,105,썬콜,650,붉켄,2024-12-19,True,True,6.190476,6.212019,650.0
1,2,오전 12:33,118,썬콜,726,붉켄,2024-12-19,True,True,6.152542,6.212019,726.0
2,13,오전 12:34,128,불독,795,협동,2024-12-19,True,True,6.210938,6.210938,795.0
3,18,오전 12:34,124,용기사,4200,블와둥,2024-12-19,True,True,33.870968,34.726694,4200.0
4,34,오전 12:35,144,허밋,2652,망용둥,2024-12-19,True,True,18.416667,18.446821,2652.0
...,...,...,...,...,...,...,...,...,...,...,...,...
453,260,오후 1:57,106,나이트,3143,듀파,2024-12-17,True,True,29.650943,30.555809,3143.0
454,260,오후 1:57,89,나이트,2800,듀파,2024-12-17,True,True,31.460674,30.555809,2800.0
455,263,오후 1:57,106,프리스트,670,불어전,2024-12-17,True,True,6.320755,6.320755,670.0
456,263,오후 1:57,107,용기사,3400,불어전,2024-12-17,True,True,31.775701,35.282984,3400.0
